In [ ]:
import socket
import os
from _thread import *
import socket
import os
import time
import random
import logging
import threading
from threading import Thread
from _thread import *
import sys
from network import *
from warehouse import *
from seller_node import *
import json
import select
import csv
from datetime import datetime
lock = threading.Lock()
logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)

In [ ]:
global_tll_value =100

In [ ]:
class Seller_thread:
    def server(self,seller_ID,server_IP):
        self.my_ID = seller_ID
        warehouse = Warehouse()
        self.items_to_sell = warehouse.get_items_to_sell(seller_ID)
        t = threading.currentThread()
        ServerSideSocket = socket.socket()
        #host = '127.0.0.5'
        host = server_IP 
        port = random_port
        ThreadCount = 0
        try:
            ServerSideSocket.bind((host, port))
        except socket.error as e:
            print(str(e))

        #print('Seller %s is listening..'%(seller_ID))
        ServerSideSocket.listen(5)

        def multi_threaded_client(connection,client_connected_to_me_IP):
            connection.send(str.encode('Server is working:'))
            while True:
                connection.setblocking(0)
                ready = select.select([connection], [], [], 0.001)
                if ready[0]:
                    data = connection.recv(2048)
                    rcvd_message = data.decode('utf-8')
                    if rcvd_message:
                        #print("seller %s this is from client %s %s"%(seller_ID,client_connected_to_me_IP,rcvd_message))
                        rcvd_message_json = json.loads(rcvd_message)
                        if rcvd_message_json:
                            #print("lets handle it")
                            response = self.rcvd_message_handler(rcvd_message_json)
                            connection.sendall(str.encode(response))
                        #print("we got this as response and sent it %s"%(response))
                #response = 'Server message: ' + data.decode('utf-8')
    #             if not data:
    #                 break
#                 if response:
#                     connection.sendall(str.encode(response))
#             connection.close()

        while True:
            Client, address = ServerSideSocket.accept()
            host, port = Client.getpeername()
            #print('seller '+seller_ID+' Connected to: ' + address[0] + ':' + str(address[1]),host)
            start_new_thread(multi_threaded_client, (Client,host))
            ThreadCount += 1
            #print('Thread Number: ' + str(ThreadCount))
        ServerSideSocket.close()

    def check_warehouse(self,product_name):
        if product_name in self.items_to_sell:
            return True
        else:
            return False
    def rcvd_message_handler(self,rcvd_message_json):
        #print(" seller %s received message %s"%(self.my_ID,rcvd_message_json))
        if rcvd_message_json["message_type"] =='announcement':
            product_name = rcvd_message_json["product_name"]
            if self.check_warehouse(product_name):
                #print(" ************** we %s have this item %s"%(self.my_ID,product_name))
                lock.acquire()
                self.items_to_sell.remove(product_name)
                lock.release()
                response_to_announcement = self.back_propagation(self.my_ID,rcvd_message_json,"back_propagation",True)
                #print("we are going to send back this %s"%(response_to_announcement))
                return response_to_announcement
            else:
                #print("oopppssss we %s do not have this! %s"%(self.my_ID,product_name))
                response_to_announcement = self.back_propagation(self.my_ID,rcvd_message_json,"back_propagation",False)
                return response_to_announcement
#         else:
#             print("oops this is not annoucement!")
           
    def back_propagation(self,seller_ID,rcvd_message_json,message_type,exist_or_not):
        ttl_value = global_tll_value
        neighbors = network.get_neighbors(seller_ID)
        product_name = rcvd_message_json["product_name"]
        buyer_ID = rcvd_message_json["buyer_ID"]
        rcvd_msg_sender_IP = rcvd_message_json["sender_IP"]
        my_IP_address = network.get_node_IP(seller_ID)
        neighbor_IP = network.get_node_IP(neighbors[0])
        if exist_or_not:
            result = "yes"
        else:
            result = "no"
        back_propagation_message = {"buyer_ID": buyer_ID ,"neighbor_IP":rcvd_msg_sender_IP, "product_name":product_name,"seller_ID":seller_ID
                       ,"message_type":message_type,"ttl_value":str(ttl_value),"sender":seller_ID,
                                   "sender_IP":my_IP_address,"exist":result} # a real dict.
        message = json.dumps(back_propagation_message)
        return message
#         lock.acquire()
#         lock.release()
#         for neighbor in neighbors:   
#             neighbor_IP = network.get_node_IP(neighbor)
#             back_propagation_message = {"buyer_ID": buyer_ID ,"neighbor_IP":neighbor_IP, "product_name":product_name,"seller_ID":seller_ID
#                        ,"message_type":message_type,"ttl_value":str(ttl_value),"sender":seller_ID} # a real dict.
#             message = json.dumps(back_propagation_message)
#             print("we as %s are going to back_propagate this %s to %s"%(seller_ID,message,neighbor_IP))
            #global_out_going_queue.append(message)
            
        
            
    def client(self,seller_ID,server_IP):
        t = threading.currentThread()
        ClientMultiSocket = socket.socket()
        #host = '127.0.0.5'
        host = server_IP
        port = random_port

        #print('Waiting for connection response')
        try:
            ClientMultiSocket.connect((host, port))
        except socket.error as e:
            print("got error for client to connect to ",buyer_ID,server_IP,str(e))
            
#         try:
#             if self.queue:
#                 donothing = True
#         except:
#             self.queue = []
        network = Network()
        neighbors = network.get_neighbors(seller_ID)
        while True:
            #time.sleep(4)
            lock.acquire()
            if global_out_going_queue:
                tmp_queue = []
                for message in global_out_going_queue:
                    tmp_queue.append(message)
                for message in tmp_queue:
                    json_message = json.loads(message) 
                    #print("this is the message from queue ",message,type(message),json_message)
                    if json_message['message_type'] == 'announcement' and json_message['neighbor_IP'] == server_IP and json_message['sender'] == seller_ID:
                #         Input = input('Hey there: ')
                        my_IP_address = network.get_node_IP(self.my_ID)
                        json_message['neighbor_IP'] =my_IP_address 
                        Input = "announce sent by "+seller_ID
                        #print(" we %s are going to announce to %s incomplete "%(seller_ID,server_IP))
            lock.release()
            try:
                ClientMultiSocket.setblocking(0)
                ready = select.select([ClientMultiSocket], [], [], 0.001)
                if ready[0]:
                    res = ClientMultiSocket.recv(1024)
                    if  "message_type" in res:
                        rcvd_message_json = json.loads(res)
                        if rcvd_message_json:
                            #print("lets handle it")
                            response = self.rcvd_message_handler(rcvd_message_json)
                            if response:
                                ClientMultiSocket.send(str.encode(response))
            except:
                pass
            
            
            
            
    def run_seller(self, seller_ID,seller_IP,server_IDs,server_client):
        if server_client =='server':
            t = threading.currentThread()
            t1 = threading.Thread(target=self.server,args = ([seller_ID,seller_IP]))
            t1.setDaemon(True)
            t1.start()
        else:
            for server in server_IDs:
                server_IP = each_node_IP[server]
                t2 = threading.Thread(target=self.client,args = ([seller_ID,server_IP]))
                t2.setDaemon(True)
                t2.start()

In [ ]:
global_out_going_queue = []
each_buyer_ongoing_item = {'test':{'test':True}}
each_buyer_announced_messages = {'test':{'test':'test'}}


In [ ]:
class Buyer:
    def initializaion(self):
        self.queue = []
        self.queue.append("fish")
        self.queue.append("negotiate")
    def buyer_server(self,buyer_ID,buyer_server_IP):
        self.my_ID = buyer_ID
        t = threading.currentThread()
        ServerSideSocket = socket.socket()
        #host = '127.0.0.5'
        host = buyer_server_IP 
        port = random_port
        ThreadCount = 0
        #print("this buyer %s wants to establish a server as %s"%(buyer_ID,buyer_server_IP))
        try:
            ServerSideSocket.bind((host, port))
        except socket.error as e:
            print(str(e))

        #print('buyer server is listening.. on %s for %s'%(buyer_server_IP,buyer_ID))
        ServerSideSocket.listen(5)

        def multi_threaded_client(connection,client_connected_to_me_IP):
            network = Network()
            connection.send(str.encode('Server is working:'))
            while True:
                rcvd_message = connection.recv(2048)
                if rcvd_message:
                    #print("buyer %s this is from client %s %s"%(buyer_ID,client_connected_to_me_IP,rcvd_message.decode('utf-8')))
                    rcvd_message_json = json.loads(rcvd_message)
                    if rcvd_message_json:
                        self.rcvd_message_handler(buyer_ID,rcvd_message_json)
#                 try:
#                     if self.queue:
#                         nothing = True
#                 except:
#                     self.queue = []
                
                if global_out_going_queue:
                    lock.acquire()
                    tmp_queue = []

                    for message in global_out_going_queue:
                        tmp_queue.append(message)
                    if len(tmp_queue)>0:
                        #print(" we %s client_connected_to_me_IP %s have these items in our queue %s  "%(buyer_ID,client_connected_to_me_IP,tmp_queue))
                        for message in tmp_queue:
                            json_message = json.loads(message) 
                            msg_buyer_ID = json_message['buyer_ID']
                            msg_product_name = json_message['product_name']
                            #print("this is the message from queue ",message,type(message),json_message)
                            #print("connection is ",connection)
                            if json_message['message_type'] == 'announcement' and json_message['neighbor_IP'] == client_connected_to_me_IP and json_message['sender']== buyer_ID:
                        #         Input = input('Hey there: ')
                                Input = "announce sent by "+buyer_ID
                                #print(" we %s are going to announce to %s "%(buyer_ID,server_IP))
                                
                                my_IP_address = network.get_node_IP(buyer_ID)
                                json_message['neighbor_IP'] = my_IP_address
                                
                                #ClientMultiSocket.send(str.encode(message))
                                new_dumped_message = json.dumps(json_message)
                                connection.sendall(str.encode(new_dumped_message))
                                #print("this is the queue for buyerID %s, %s",buyer_ID,self.queue)
                                global_out_going_queue.remove(message)
                            elif json_message['message_type'] == 'back_propagation' and json_message['neighbor_IP'] == client_connected_to_me_IP and json_message['sender']== buyer_ID: 
                                #print("back_propagation sent from ",json_message["seller_ID"])
                                #ClientMultiSocket.send(str.encode(message))
                                connection.sendall(str.encode(message))
                                #print("this is the queue for buyerID %s, %s",buyer_ID,self.queue)
                                global_out_going_queue.remove(message)
                    lock.release()
                else:
                    response = 'Server message: ' + rcvd_message.decode('utf-8')
    #                 if not data:
    #                     break
                    connection.sendall(str.encode(buyer_ID))
#             connection.close()

        while True:
            Client, address = ServerSideSocket.accept()
            host, port = Client.getpeername()
            #print(buyer_ID+' Connected to: ' + address[0] + ':' + str(address[1]),host)
            start_new_thread(multi_threaded_client, (Client,host ))
            ThreadCount += 1
            #print('Thread Number: ' + str(ThreadCount))
        ServerSideSocket.close()        
        
    def check_ttl(self,ttl_value):
        if int(ttl_value)>0:
            return True
        else:
            return False
        
    def rcvd_message_handler(self,buyer_ID,rcvd_message_json):
        #print(" buyer %s received message %s"%(buyer_ID,rcvd_message_json))
        ttl_value = rcvd_message_json["ttl_value"]
        if rcvd_message_json["message_type"] =='announcement':
            product_name = rcvd_message_json["product_name"]
            
            rcvd_buyer_ID = rcvd_message_json["buyer_ID"]
            rcvd_neighbor_IP = rcvd_message_json["sender_IP"]
            
            key = rcvd_buyer_ID+','+product_name
            if buyer_ID in each_buyer_announced_messages:
                if key not in each_buyer_announced_messages[buyer_ID]:
                    try:
                        each_buyer_announced_messages[buyer_ID][key].append(rcvd_neighbor_IP)
                    except:
                        each_buyer_announced_messages[buyer_ID][key]= []
                        each_buyer_announced_messages[buyer_ID][key].append(rcvd_neighbor_IP)
                else:
                    if received_from_neighbor_IP not in each_buyer_announced_messages[buyer_ID][key]:
                        each_buyer_announced_messages[buyer_ID][key].append(rcvd_neighbor_IP)

            else:
                each_buyer_announced_messages[buyer_ID] = {}
                each_buyer_announced_messages[buyer_ID][key]= []
                each_buyer_announced_messages[buyer_ID][key].append(rcvd_neighbor_IP)

            
            
            network = Network()
            if self.check_ttl(ttl_value):
                if rcvd_buyer_ID != buyer_ID:
                    #print("***** we %s are going to announce to other neighbors rcvd_buyer_ID %s"%(self.my_ID,rcvd_buyer_ID))
                    lock.acquire()
#                     self.queue = []
                    ttl_value = global_tll_value
                    ttl_value = int(ttl_value)-1
                    neighbors = network.get_neighbors(self.my_ID)
                    my_IP_address = network.get_node_IP(buyer_ID)
                    for neighbor in neighbors:
                        neighbor_IP = network.get_node_IP(neighbor)
                        #print("for neighbor %s,its IP %s"%(neighbor,neighbor_IP))
                        print(neighbor_IP , rcvd_neighbor_IP , neighbor , rcvd_buyer_ID)
                        if neighbor_IP != rcvd_neighbor_IP and neighbor != rcvd_buyer_ID:
                            announce = {"buyer_ID": rcvd_buyer_ID ,"neighbor_IP":neighbor_IP, "product_name":product_name,
                                       "message_type":"announcement","ttl_value":str(ttl_value),"sender":self.my_ID,
                                       "sender_IP":my_IP_address} # a real dict.
                            #print("so we %s are adding this to the queue %s to be sent to %s"%(self.my_ID,announce,neighbor_IP))
                            message = json.dumps(announce)

                            global_out_going_queue.append(message)
                               
                    lock.release()
        elif rcvd_message_json["message_type"] =='back_propagation':
            product_name = rcvd_message_json["product_name"]
            rec_seller_ID = rcvd_message_json["seller_ID"]
            if buyer_ID in each_buyer_ongoing_item:
                if product_name in each_buyer_ongoing_item[buyer_ID]:
                    if each_buyer_ongoing_item[buyer_ID][product_name]:
                        if rcvd_message_json["exist"]=="yes":
                            print("*********** we %s  were going to buy this product %s. we got it from %s"%(buyer_ID,product_name,rec_seller_ID))
                            each_buyer_ongoing_item[buyer_ID][product_name] = False
                        else:
                            print("*********** we %s  were going to buy this product %s. we did not get it from %s"%(buyer_ID,product_name,rec_seller_ID))
            else:
                lock.acquire()
                ttl_value = int(ttl_value)-1
                network = Network()
                neighbors = network.get_neighbors(buyer_ID)
                rcvd_buyer_ID = rcvd_message_json["buyer_ID"]
                rcvd_seller_ID = rcvd_message_json["seller_ID"]
                rcvd_sender_IP = rcvd_message_json["sender_IP"]
                result = rcvd_message_json["exist"]
                key = rcvd_buyer_ID+','+product_name
                my_IP_address = network.get_node_IP(buyer_ID)
                if buyer_ID in each_buyer_announced_messages:
                    #print("we %s are going to add these to the queue"%(buyer_ID))
                    if key in each_buyer_announced_messages[buyer_ID]:
                        for neighbor_IP in each_buyer_announced_messages[buyer_ID][key]:   
                            #print("for neighbor ",neighbor_IP)
                            if neighbor_IP != rcvd_sender_IP and neighbor_IP !=my_IP_address:
                                back_propagation_message = {"buyer_ID": rcvd_buyer_ID ,"neighbor_IP":neighbor_IP, "product_name":product_name,
                                           "message_type":"back_propagation","ttl_value":str(ttl_value),"sender":buyer_ID,"seller_ID":rcvd_seller_ID,
                                                           "sender_IP":my_IP_address,"exist":result} # a real dict.
                                message = json.dumps(back_propagation_message)
                                #print("we as %s are going to back_propagation this %s to %s"%(buyer_ID,message,neighbor_IP))
                                global_out_going_queue.append(message)
                    each_buyer_announced_messages[buyer_ID] = {}
                lock.release()
                
    def client(self,buyer_ID,server_IP):
        t = threading.currentThread()
        ClientMultiSocket = socket.socket()
        #host = '127.0.0.5'
        host = server_IP
        port = random_port

        #print('Waiting for connection response')
        try:
            ClientMultiSocket.connect((host, port))
        except socket.error as e:
            print("got error for client to connect to ",buyer_ID,server_IP,str(e))

        #res = ClientMultiSocket.recv(1024)
        network = Network()
        neighbors = network.get_neighbors(buyer_ID)
        while True:
            #time.sleep(4)
            if len(global_out_going_queue)>0:
                
                tmp_queue = []
                
                for message in global_out_going_queue:
                    tmp_queue.append(message)
                if len(tmp_queue)>0:
                    lock.acquire()
                    #print("as a buyer %s and the client of a seller %s this the message from queue %s"%(buyer_ID,server_IP,tmp_queue))
                    for message in tmp_queue:
                        json_message = json.loads(message) 
                        #print("for message ",json_message)
                        if json_message['message_type'] == 'announcement' and json_message['neighbor_IP'] == server_IP and json_message['sender'] == buyer_ID:
                    #         Input = input('Hey there: ')
                            my_IP_address = network.get_node_IP(buyer_ID)
                            json_message['neighbor_IP'] = my_IP_address 
                            json_message['sender'] = buyer_ID 
                            Input = "announce sent by "+buyer_ID
                            #print(" we %s are going to announce to %s "%(buyer_ID,server_IP))
                            msg_buyer_ID = json_message['buyer_ID']
                            msg_product_name = json_message['product_name']
                            msg_sender_IP = json_message['sender_IP']
                            key = msg_buyer_ID+','+msg_product_name
                            if buyer_ID in each_buyer_announced_messages:
                                try:
                                    each_buyer_announced_messages[buyer_ID][key].append(msg_sender_IP)
                                except:
                                    each_buyer_announced_messages[buyer_ID][key]= []
                                    each_buyer_announced_messages[buyer_ID][key].append(msg_sender_IP)
                            else:
                                each_buyer_announced_messages[buyer_ID] = {}
                                each_buyer_announced_messages[buyer_ID][key]= []
                                each_buyer_announced_messages[buyer_ID][key].append(msg_sender_IP)

                            new_dumped_message = json.dumps(json_message)
                            ClientMultiSocket.send(str.encode(new_dumped_message))
                            #print("this is the queue for buyerID %s, %s",buyer_ID,self.queue)
                            global_out_going_queue.remove(message)
                        elif json_message['message_type'] == 'back_propagation' and json_message['neighbor_IP'] == server_IP and json_message['sender'] == buyer_ID: 
                            Input = "back_propagation sent from "+json_message['seller_ID']
                            ClientMultiSocket.send(str.encode(message))
                            #print("this is the queue for buyerID %s, %s",buyer_ID,self.queue)
                            global_out_going_queue.remove(message)
                        #print("done!")
                        
                    lock.release()
            
            ClientMultiSocket.setblocking(0)
            ready = select.select([ClientMultiSocket], [], [], 0.001)
            if ready[0]:
                res = ClientMultiSocket.recv(1024)
                #print("thsi is from client function of buyer ",res)
                if "message_type" in res:
                    rcvd_message_json = json.loads(res)
                    if rcvd_message_json:
                        self.rcvd_message_handler(buyer_ID,rcvd_message_json)
#             else:
#                 print("as a buyer %s and the client of a seller %s  queue is empty "%(buyer_ID,server_IP))
                #time.sleep(1)
#             res = ClientMultiSocket.recv(1024)
#             rcvd_message_json = json.loads(res)
#             if rcvd_message_json:
#                 self.rcvd_message_handler(rcvd_message_json,server_IP)

        
            #print("as a buyer %s and the client of a seller %s we received this %s from it "%(buyer_ID,server_IP,res.decode('utf-8')))
        ClientMultiSocket.close()
    def run_buyer(self, buyer_ID,buyer_IP,server_IDs,server_client):
        if server_client =='server':
            t = threading.currentThread()
            t1 = threading.Thread(target=self.buyer_server,args = ([buyer_ID,buyer_IP]))
            t1.setDaemon(True)
            t1.start()
        else:
            for server in server_IDs:
                server_IP = each_node_IP[server]
                t2 = threading.Thread(target=self.client,args = ([buyer_ID,server_IP]))
                t2.setDaemon(True)
                t2.start()
            time.sleep(2)
            
            self.queue = []
            warehouse = Warehouse()
            network = Network()
            buying_items = warehouse.get_required_items(buyer_ID)
            neighbors = network.get_neighbors(buyer_ID)
            my_IP_address = network.get_node_IP(buyer_ID)
            for item in buying_items:
                start = datetime.now()
                start = round(time.time() * 1000)
                lock.acquire()
                self.ongoing_shopping = True
                try:
                    each_buyer_ongoing_item[buyer_ID][item] = True
                except:
                    each_buyer_ongoing_item[buyer_ID] = {}
                    each_buyer_ongoing_item[buyer_ID][item] = True
                self.queue = []
                ttl_value = global_tll_value
                for neighbor in neighbors:   
                    neighbor_IP = network.get_node_IP(neighbor)
                    announce = {"buyer_ID": buyer_ID ,"neighbor_IP":neighbor_IP, "product_name":item,
                               "message_type":"announcement","ttl_value":str(ttl_value),"sender":buyer_ID,
                               "sender_IP":my_IP_address} # a real dict.
                    message = json.dumps(announce)
                    #print("we as %s are going to announce this %s to %s"%(buyer_ID,message,neighbor_IP))
                    global_out_going_queue.append(message)
                lock.release()
                while(each_buyer_ongoing_item[buyer_ID][item]):
                    waiting = True
                end = datetime.now()
                end = round(time.time() * 1000)
                print("*********** as the buyer %s we were going to buy this product %s in %s"%(buyer_ID,item,str(end-start)))
                each_chain_length_transaction_times = "each_chain_length_transaction_times.csv"
                with open(each_chain_length_transaction_times, 'a') as newFile:
                    newFileWriter = csv.writer(newFile)
                    newFileWriter.writerow([chain_lenght,str(end-start)])  
    
    def do_shopping(self,buyer_ID,buyer_IP,server_IDs,server_client):
        self.queue = []
        self.queue.append("fish")
        self.queue.append("negotiate to 127.0.0.2")
        
#         main_thread = threading.current_thread()
#         for t in threading.enumerate():
#             if t is main_thread:
#                 continue
#         logging.debug('joining %s', t.getName())
#         t.join()
        

In [ ]:
node_role_values = {'1':'client','2':'client','3':'client','4':'server'}
node_role_values = {'1':'client','2':'client','4':'client','5':'server'}
node_role_values = {'1':'client','2':'client','4':'client','5':'client',
                   '6':'client','7':'client','8':'client','9':'client','10':'client',
                   '11':'client','12':'client','14':'client','15':'client',
                   '16':'client','17':'client','18':'client','19':'client','20':'server'}

each_node_IP = {'1':'127.0.0.1',
               '2':'127.0.0.2',
               '3':'127.0.0.3',
               '4':'127.0.0.4',
               '5':'127.0.0.5',
               '5':'127.0.0.5',
               '6':'127.0.0.5',
               '7':'127.0.0.5',
               '8':'127.0.0.5',
               '9':'127.0.0.5',
               '10':'127.0.0.5',
               '11':'127.0.0.5'}
each_node_neighbors = {
                        '1':['2'],
                        '2':['1','4'],
                        '4':['2','5'],
                        '5':['4']
}
each_node_neighbors = {
                        '1':['2'],
                        '2':['1','4'],
                         '4':['2','5'],
                        '5':['4']}


network = Network()
for node in node_role_values:
    each_node_neighbors[node] = network.get_neighbors(node)
for node in node_role_values:
    each_node_IP[node] = network.get_node_IP(node)   
random_port = random.randint(2000,4000)
chain_lenght = 20

for item,role in node_role_values.items():
    #time.sleep(2)
    if role =='server':
        seller = Seller_thread()
        t1 = threading.Thread(target=seller.run_seller,args = (item,each_node_IP[item],each_node_neighbors[item],'server'))
        t1.setDaemon(True)
        t1.start()
    else:
        buyer = Buyer()
        t2 = threading.Thread(target=buyer.run_buyer,args = (item,each_node_IP[item],each_node_neighbors[item],'server'))
        t2.setDaemon(True)
        t2.start()
time.sleep(2)
for item,role in node_role_values.items():
    
    if role =='server':
        t3 = threading.Thread(target=seller.run_seller,args = (item,each_node_IP[item],each_node_neighbors[item],'client'))
        t3.setDaemon(True)
        t3.start()
    if role !='server':
        buyer = Buyer()
        t4 = threading.Thread(target=buyer.run_buyer,args = (item,each_node_IP[item],each_node_neighbors[item],'client'))
        t4.setDaemon(True)
        t4.start()
        
        
# for item,role in node_role_values.items():
#     time.sleep(2)
#     if role !='server':
#         buyer = Buyer()
#         t3 = threading.Thread(target=buyer.do_shopping,args = (item,each_node_IP[item],each_node_neighbors[item],'client'))
#         t3.setDaemon(True)
#         t3.start()
        
main_thread = threading.current_thread()
for t in threading.enumerate():
    if t is main_thread:
        continue
logging.debug('joining %s', t.getName())
t.join()